<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title"><b>A Magic Square Solver</b></span> by <a xmlns:cc="http://creativecommons.org/ns#" href="http://mate.unipv.it/gualandi" property="cc:attributionName" rel="cc:attributionURL">Stefano Gualandi</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/mathcoding/opt4ds" rel="dct:source">https://github.com/mathcoding/opt4ds</a>.

**NOTE:** Run the following script whenever running this script on a Google Colab.

In [ ]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("glpk") or os.path.isfile("glpk")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk 
        except:
            pass

# Magic Square Solver
In this notebook, we propose an ILP model to the [Magic Square](https://en.wikipedia.org/wiki/Magic_square) puzzle.

The puzzle asks to place into a grid of size $n \times n$ the digits from $1$ to $n^2$, in such a way that the sum of the digits in each row, the sum of digits in each column, and the sum of the digits on the two main diagonals, is equal to the same number.

## ILP Model
The model we propose is as follows.

**Decision Variables:** We use two type of variables:

* The variable $x_{ijk} \in \{0,1\}$ is equal to 1 if the cell in position $(i,j)$ contains the digit $k$, and it is equal to 0 otherwise. For easy of exposition, we use the set $I,J:=\{1,\dots,n\}$ and $K := \{1,\dots,n^2\}$.
* The variable $z\in\mathbb{Z}_+$ represents the magic number.

**Objective function:** Since the problem is a feasibility problem, we can set the objective function equal to a constant value. Otherwise, we can add the sum of every variable (this way we avoid also a warning from the solver).

**Constraints:** We introduce the following linear constraints, which encode the puzzle rules:

1. Every digit, we can be placed into a single position:
$$
    \sum_{i \in I}\sum_{j \in J} x_{ijk} = 1, \;\; \forall k \in K
$$
2. In every position, we can place a single digit:
$$
    \sum_{k \in K} x_{ijk} = 1, \;\; \forall i \in I, \; \forall j \in J
$$
3. The sum of the digits in each row must be equal to $z$:
$$
    \sum_{j \in J}\sum_{k \in K} k x_{ijk} = z, \;\; \forall i \in I
$$
3. The sum of the digits in each column must be equal to $z$:
$$
    \sum_{i \in I}\sum_{k \in K} k x_{ijk} = z, \;\; \forall j \in J
$$
4. The sum of the digits over the two main diagonals is equal to $z$:
$$
    \sum_{i \in I} \sum_{k \in K} x_{iik} = z,
$$
$$
    \sum_{i \in I} \sum_{k \in K} x_{i(n-i+1)k} = z,
$$

We show next how to implement this model in Pyomo.

## Pyomo implementation
As a first step we import the Pyomo libraries.

In [ ]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory
from pyomo.environ import Binary, RangeSet, ConstraintList, PositiveIntegers

We create an instance of the  class *ConcreteModel*, and we start to add the *RangeSet* and *Var* corresponding to the index sets and the variables of our model. We set also the objective function.

In [ ]:
# Create concrete model
model = ConcreteModel()

n = 4

# Set of indices
model.I = RangeSet(1, n)
model.J = RangeSet(1, n)
model.K = RangeSet(1, n*n)

# Variables
model.z = Var(within=PositiveIntegers)
model.x = Var(model.I, model.J, model.K, within=Binary)

# Objective Function
model.obj = Objective(expr = model.z)

Then, we encode all the constraints of our model using the Pyomo syntax.

In [ ]:
def Unique(model, k):
    return sum(model.x[i,j,k] for j in model.J for i in model.I) == 1
model.unique = Constraint(model.K, rule = Unique)

def CellUnique(model, i, j):
    return sum(model.x[i,j,k] for k in model.K) == 1
model.cellUnique = Constraint(model.I, model.J, rule = CellUnique)

def Row(model, i):
    return sum(k*model.x[i,j,k] for j in model.J for k in model.K) == model.z
model.row = Constraint(model.I, rule = Row)

def Col(model, j):
    return sum(k*model.x[i,j,k] for i in model.I for k in model.K) == model.z
model.column = Constraint(model.J, rule = Col)

model.diag1 = Constraint(
    expr = sum(k*model.x[i,i,k] for i in model.I for k in model.K) == model.z)

model.diag2 = Constraint(
    expr = sum(k*model.x[i,n-i+1,k] for i in model.I for k in model.K) == model.z)

Finally, we solve the model for a given $n$ and we check the solution status.

In [ ]:
# Solve the model
sol = SolverFactory('glpk').solve(model)

# CHECK SOLUTION STATUS

# Get a JSON representation of the solution
sol_json = sol.json_repn()
# Check solution status
if sol_json['Solver'][0]['Status'] != 'ok':
    print("Problem unsolved")
if sol_json['Solver'][0]['Termination condition'] != 'optimal':
    print("Problem unsolved")

If the problem is solved and a feasible solution is available, we write the solution into a colorful **magic square**.

In [ ]:
def PlotMagicSquare(x, n):
    # Report solution value
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools
    
    sol = np.zeros((n,n), dtype=int)
    
    for i, j, k in x:
        if x[i,j,k]() > 0.5:
            sol[i-1,j-1] = k
    
    cmap = plt.get_cmap('Blues')
    
    plt.figure(figsize=(6,6))
    plt.imshow(sol, interpolation='nearest', cmap=cmap)
    plt.title("Magic Square, Size: {}".format(n))
    plt.axis('off')
    
    for i, j in itertools.product(range(n), range(n)):
        plt.text(j, i, "{:d}".format(sol[i, j]), 
                 fontsize=24, ha='center', va='center')
            
    plt.tight_layout()
    plt.show()
    
PlotMagicSquare(model.x, n)